In [1]:
import torch
import numpy as np
from stable_baselines3 import PPO


import os, shutil, sys
sys.path.append(os.path.abspath('./env'))
sys.path.append(os.path.abspath('./common'))

from env.env_move_sector_v2  import HumanMoveSectorActionV2
from env.env_move_sector_v3  import HumanMoveSectorActionV3
from env.env_move_ray_v3  import HumanMoveRayActionV3
from env.env_move_fast_sector  import HumanMoveFastSectorAction

import common.mlflow_sb3_helper as mlf
import common.tensorboard_sb3_helper as tnb
USE_MLFLOW = False

from pytz import timezone
from datetime import datetime

TZ = timezone('Europe/Moscow')
device = "cuda" if torch.cuda.is_available() else "cpu"

activations = {'ReLU':torch.nn.ReLU, 'Tanh':torch.nn.Tanh}

In [2]:
exp_params = {
    'env_name': '',
    'algorithm_name': '',
    'exp_id': 0,
    'exp_name': f'exp_{datetime.now(TZ).strftime("%d%m_%H%M%S")}',
    'seed': 21,
    'net': {
        'activation': 'Tanh',
        'pi': [512, 512, 512],
        'vf': [512, 512, 512],
    },
    'training': {
        'iteration_count': 2,
        'episode_count': 39000,
        'policy': 'MlpPolicy',
        'learning_rate': 0.001,
        'buffer_size': 1500000,
        'learning_starts': 100,
        'batch_size': 1024,
        'gamma': 0.99,
        'verbose': 0,
        'device': device,
    },
    'options':{
        'finish_dist':100,
        'start_dist':110,
        'delta_phi':0
    },
    'validation': {
        'validate_agent_every_n_eps': 20000,
        'log_interval': 10
    },
}

policy_kwargs = dict(activation_fn = activations[exp_params['net']['activation']],
                     net_arch = dict(pi=exp_params['net']['pi'], vf=exp_params['net']['vf']))



In [3]:
log_helper = {}
log_path = {}

if USE_MLFLOW:
    log_helper = mlf.MLflowServerHelper("http://192.168.0.206:2670", False)
else:
    log_helper = tnb.TensorboardHelper("",False)

In [4]:
experiment_id = 623
log_path = {
 'PPO': f'mlflow-artifacts:/{experiment_id}/2363066ef4dd42ed8d8b1d8e152ff8fe/artifacts/env_MoveRay3_IgnoreObst_exp_1211_201942/sb3/model.zip'
 }

In [ ]:

load_path = os.path.join(str(experiment_id))
if  os.path.isdir(load_path) == False:
    os.mkdir(load_path)
load_path

In [ ]:
loadel_models = {}

for model_name, path in log_path.items():
    loadel_models[model_name] =  f'{load_path}/{model_name}/'
    log_helper.load_artifact(path,loadel_models[model_name])

In [7]:
env = HumanMoveRayActionV3(target_point_rand=True, object_ignore=True)
env_render = HumanMoveRayActionV3(target_point_rand=True, object_ignore=True, render_mode='rgb_array')

In [ ]:
exp_params['env_name'] = env.name()
exp_name = 'env_' + exp_params['env_name'] + '_' + exp_params['exp_name']
experiment_id = log_helper.new_experiment(exp_name)
exp_params['exp_id'] = experiment_id
print(exp_name)

In [ ]:
#!!! Дорасчет в СТАРОМ эксперименте
exp_name = log_helper.get_experiment(experiment_id)
print(exp_name)
exp_params['exp_id'] = experiment_id
exp_params['env_name'] = env.name()

In [ ]:

exp_params['algorithm_name'] = 'PPO'

path = loadel_models['PPO']
print(path)
name = 'PPO'

b_first = True

for i in range(exp_params['training']['iteration_count']):


    exp_params['seed'] = int(datetime.now(TZ).strftime("%H%M%S"))

    if b_first == True:
        b_first = False
        model = PPO.load(f'{path}/model.zip', env=env, device=device)
    else:
        print(exp_name)
        model = PPO.load(f'{exp_name}/model.zip', env=env, device=device)


    art_loc, exp_name, run_id = log_helper.learn_and_fix(
                                model=model, 
                                env= env_render,
                                run_name=f'{name}_{i+1}',
                                episode_count = exp_params['training']['episode_count'],
                                parameters=exp_params,
                                experiment_id=experiment_id,
                                checkpoint_interval=exp_params['validation']['validate_agent_every_n_eps'],
                                log_interval=exp_params['validation']['log_interval'],
                                video_episods=3,
                                video_fps=120
                                )

    log_path[name] = f'{art_loc}/{run_id}/artifacts/{exp_name}/sb3/model.zip'



In [ ]:
log_path

In [ ]:
shutil.rmtree(os.path.join(load_path))